In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn import Module
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import Optimizer
from typing import Tuple, Union, Callable
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('Electricity Dataset.csv')
print(df.columns)

Index(['site area', 'water consumption', 'recycling rate', 'utilisation rate',
       'air qality index', 'issue reolution time', 'resident count',
       'electricity cost', 'electricity cost per resident',
       'water_consumption_per_resident', 'recycling_efficency_rate',
       'resource_intensity', 'resolution_efficiency', 'air_quality_impact',
       'energy_efficiency', 'water_electricity_ratio', 'population_density',
       'issue_per_utility', 'sustainability_score',
       'structure type_Commercial', 'structure type_Industrial',
       'structure type_Mixed-use', 'structure type_Residential'],
      dtype='object')


In [3]:
features = df.drop('electricity cost', axis=1).values
targets = df['electricity cost'].values

In [4]:
feature_scaler = StandardScaler()
features = feature_scaler.fit_transform(features)

target_scaler = StandardScaler()
targets = target_scaler.fit_transform(targets.reshape(-1, 1)).flatten()

In [5]:
class ElectricityDataset(Dataset):
    def __init__(self, features: Union[Tensor, list, np.ndarray], targets: Union[Tensor, list, np.ndarray]) -> None:
        self.features: Tensor = torch.tensor(features, dtype=torch.float32)
        self.targets: Tensor = torch.tensor(targets, dtype=torch.float32)

    def __len__(self) -> int:
        return len(self.features)

    def __getitem__(self, idx: int) -> Tuple[Tensor, Tensor]:
        return self.features[idx], self.targets[idx]

In [6]:
dataset = ElectricityDataset(features, targets)

In [7]:
total_size = len(dataset)

train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size 

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [8]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
data_iter = iter(train_loader)
features_batch, targets_batch = next(data_iter)


print("Features batch shape:", features_batch.shape)
print("Targets batch shape:", targets_batch.shape)
print("First feature sample:", features_batch[0])
print("First target sample:", targets_batch[0])

Features batch shape: torch.Size([32, 22])
Targets batch shape: torch.Size([32])
First feature sample: tensor([ 0.4240, -0.3637, -0.8792,  0.6929, -0.9389,  0.0775,  0.4119, -0.8734,
        -0.8715, -0.4902, -1.1074, -0.8822, -0.2058, -1.0549, -0.5087,  0.2312,
        -0.0520, -0.1224, -0.6554, -0.3341,  1.9681, -0.8062])
First target sample: tensor(0.0173)


In [10]:
class FFNRegressor(nn.Module):
    def __init__(self, input_dim: int) -> None:
        super(FFNRegressor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)  # No activation for regression output
        )

    def forward(self, x: Tensor) -> Tensor:
        return self.network(x).squeeze(1)

In [11]:
def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    loss_fn: Callable[[Tensor, Tensor], Tensor],
    optimizer: Optimizer,
    epochs: int,
    device: torch.device = torch.device("cpu")
) -> None:
    model.to(device)

    for epoch in range(epochs):
        model.train()
        running_train_loss = 0.0
        train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

        for batch_features, batch_targets in train_loop:
            batch_features = batch_features.to(device)
            batch_targets = batch_targets.to(device)

            # Forward pass
            outputs = model(batch_features)
            loss = loss_fn(outputs, batch_targets)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()
            train_loop.set_postfix(train_loss=loss.item())

        avg_train_loss = running_train_loss / len(train_loader)

        # ---- Validation Loop ----
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for val_features, val_targets in val_loader:
                val_features = val_features.to(device)
                val_targets = val_targets.to(device)

                val_outputs = model(val_features)
                val_loss = loss_fn(val_outputs, val_targets)
                running_val_loss += val_loss.item()

        avg_val_loss = running_val_loss / len(val_loader)

        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f} | Val Loss = {avg_val_loss:.4f}")


In [12]:
def predict_model(
    model: nn.Module,
    data_loader: DataLoader,
    device: torch.device = torch.device("cpu")
) -> Tensor:
    """
    Generates predictions from a trained model.

    Args:
        model (nn.Module): Trained PyTorch model.
        data_loader (DataLoader): DataLoader for input features.
        device (torch.device): Device to run predictions on. Default is CPU.

    Returns:
        Tensor: Predictions concatenated across all batches.
    """
    model.eval()
    model.to(device)
    predictions = []

    with torch.no_grad():
        for features, _ in data_loader:  # targets not needed for prediction
            features = features.to(device)
            outputs = model(features)
            predictions.append(outputs.cpu())

    return torch.cat(predictions, dim=0)


In [13]:
def save_model(model: Module, filename: Union[str, None] = "model.pth") -> None:
    """
    Saves a PyTorch model to a specified file.

    Args:
        model (Module): The trained PyTorch model to save.
        filename (str): The filename to save the model to. Defaults to 'model.pth'.
    """
    torch.save(model.state_dict(), filename)
    print(f"Model saved to: {filename}")

In [14]:
def load_model(model_class: type[Module], input_dim: int, filename: Union[str, None] = "model.pth") -> Module:
    """
    Loads a model from a specified file and returns the model with weights loaded.

    Args:
        model_class (type[Module]): The model class to instantiate.
        input_dim (int): The input dimension for the model constructor.
        filename (str): The filename to load the model from. Defaults to 'model.pth'.

    Returns:
        Module: The model with loaded weights.
    """
    model = model_class(input_dim)
    model.load_state_dict(torch.load(filename))
    model.eval()
    print(f"Model loaded from: {filename}")
    return model

In [15]:
model = FFNRegressor(input_dim=features.shape[1])
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=10
)

Epoch 1: Train Loss = 0.1673 | Val Loss = 0.0181


Epoch 2: Train Loss = 0.0129 | Val Loss = 0.0093


Epoch 3: Train Loss = 0.0067 | Val Loss = 0.0052


Epoch 4: Train Loss = 0.0046 | Val Loss = 0.0039


Epoch 5: Train Loss = 0.0037 | Val Loss = 0.0041


Epoch 6: Train Loss = 0.0030 | Val Loss = 0.0031


Epoch 7: Train Loss = 0.0028 | Val Loss = 0.0031


Epoch 8: Train Loss = 0.0023 | Val Loss = 0.0028


Epoch 9: Train Loss = 0.0022 | Val Loss = 0.0024


Epoch 10: Train Loss = 0.0020 | Val Loss = 0.0024
